# How to Create Subscriptions

> Set up recurring payments for services

This guide shows you how to create recurring subscription payments using FastStripe.

## Problem

You want to charge customers on a recurring basis (monthly, yearly, etc.) for a service or product.

## Solution

Use FastStripe's `subscription()` method which creates the necessary recurring price and checkout session.

In [ ]:
from faststripe.core import StripeApi
from faststripe.page import pages
import os

sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])

## Monthly subscription

In [ ]:
# Create a monthly subscription
subscription = sapi.subscription(
    product_name='Pro Plan',
    amount_cents=1999,  # $19.99/month
    success_url='https://yoursite.com/welcome',
    cancel_url='https://yoursite.com/pricing',
    interval='month'
)

print(f"Subscription URL: {subscription.url}")

Subscription URL: https://billing.answer.ai/c/pay/cs_test_a1Koczt2muVRPSvhdKJvW72BhK0tyqULhDdvdNJLzWH2F1F7VRKeSxXqDb#fidkdWxOYHwnPyd1blpxYHZxWjA0VDFNMTFOQm10THI8VV1oX0xtbUt8R0RvX3RicGtMXWBKVm1oN01uYTF8VXZBSXRVSTVxazNqcUcxXHc0V1FGZEpdbDxGZElAc0BUZzNJZHVhfEZcM2tDNTV%2FSXZvYTRtTicpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl


## Annual subscription with discount

In [ ]:
# Create an annual subscription (typically discounted)
annual_sub = sapi.subscription(
    product_name='Pro Plan',
    amount_cents=19999,  # $199.99/year (save $39.89)
    success_url='https://yoursite.com/welcome',
    cancel_url='https://yoursite.com/pricing',
    interval='year'
)

print(f"Annual subscription URL: {annual_sub.url}")

Annual subscription URL: https://billing.answer.ai/c/pay/cs_test_a1nEYwo42vkexu8k6G7rcTkH7hojCqtZ8R29mI9kWEhzSAlSdFFGC5F3MS#fidkdWxOYHwnPyd1blpxYHZxWjA0VDFNMTFOQm10THI8VV1oX0xtbUt8R0RvX3RicGtMXWBKVm1oN01uYTF8VXZBSXRVSTVxazNqcUcxXHc0V1FGZEpdbDxGZElAc0BUZzNJZHVhfEZcM2tDNTV%2FSXZvYTRtTicpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl


## Pre-fill customer email

In [ ]:
# Pre-fill customer email in checkout
subscription = sapi.subscription(
    product_name='Pro Plan',
    amount_cents=1999,
    success_url='https://yoursite.com/welcome',
    cancel_url='https://yoursite.com/pricing',
    customer_email='user@example.com'
)

print(f"Subscription with email: {subscription.url}")

Subscription with email: https://billing.answer.ai/c/pay/cs_test_a1OEiShyZ519kvG30C7LjJLfTROahjVckQYOU87yaAHaxzfgl4kWJITxCv#fidkdWxOYHwnPyd1blpxYHZxWjA0VDFNMTFOQm10THI8VV1oX0xtbUt8R0RvX3RicGtMXWBKVm1oN01uYTF8VXZBSXRVSTVxazNqcUcxXHc0V1FGZEpdbDxGZElAc0BUZzNJZHVhfEZcM2tDNTV%2FSXZvYTRtTicpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl


## Manual subscription setup

For more control over subscription features:

In [ ]:
# Create product
product = sapi.products.post(
    name='Enterprise Plan',
    description='Full access to all features'
)

# Create recurring price
price = sapi.prices.post(
    product=product.id,
    unit_amount=9999,  # $99.99
    currency='usd',
    recurring={
        'interval': 'month',
        'usage_type': 'licensed'  # per-seat billing
    }
)

# Create subscription checkout
checkout = sapi.checkout.sessions_post(
    mode='subscription',
    line_items=[{
        'price': price.id,
        'quantity': 1
    }],
    success_url='https://yoursite.com/success?session_id={CHECKOUT_SESSION_ID}',
    cancel_url='https://yoursite.com/cancel',
    allow_promotion_codes=True,  # Enable promo codes
    subscription_data={
        'trial_period_days': 14  # 14-day free trial
    }
)

print(f"Enterprise subscription: {checkout.url}")

Enterprise subscription: https://billing.answer.ai/c/pay/cs_test_b12leucMZratXDH7VRToLgtx9jUGKq8HF3qbTg34Sz5RsOUEhr0Y9lC6V7#fidkdWxOYHwnPyd1blpxYHZxWjA0VDFNMTFOQm10THI8VV1oX0xtbUt8R0RvX3RicGtMXWBKVm1oN01uYTF8VXZBSXRVSTVxazNqcUcxXHc0V1FGZEpdbDxGZElAc0BUZzNJZHVhfEZcM2tDNTV%2FSXZvYTRtTicpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPydocGlxbFpscWBoJyknYGtkZ2lgVWlkZmBtamlhYHd2Jz9xd3BgeCUl


## Managing existing subscriptions

Retrieve and modify existing subscriptions:

In [ ]:
# List all subscriptions
subscriptions = pages(sapi.subscriptions.get, limit=100)
print(f"Found {len(subscriptions)} subscriptions")

# Get a specific subscription
sub = subscriptions[0]
print(f"Subscription {sub.id}: {sub.status}")

Found 288 subscriptions
Subscription sub_1Rh0UdKGhqIw9PXmDnHTOT3M: active


## Best practices

- Always use webhooks to handle subscription lifecycle events
- Consider offering free trials to increase conversions
- Use promotion codes for discounts and marketing campaigns
- Set up proper success/cancel URLs that match your user flow
- Test subscription flows thoroughly in test mode